In [1]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist



import folium
from geopy.geocoders import Nominatim

In [2]:
path = os.path.join(os.path.abspath('../data'), '2016_neighbourhood_profiles.csv')
profile_df = pd.read_csv(path)
profile_df.head(20)

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,...,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,2.90%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%
5,Population,Population and dwellings,Census Profile 98-316-X2016001,Total private dwellings,"1,179,057","9,371","8,535","4,732","18,109","12,473",...,"8,054","8,721","19,098","5,620","3,604","6,185","6,103","7,475","11,051","5,847"
6,Population,Population and dwellings,Census Profile 98-316-X2016001,Private dwellings occupied by usual residents,"1,112,929","9,120","8,136","4,616","15,934","12,124",...,"7,549","8,509","18,436","5,454","3,449","5,887","5,676","7,012","10,170","5,344"
7,Population,Population and dwellings,Census Profile 98-316-X2016001,Population density per square kilometre,"4,334","3,929","3,034","2,435","10,863","2,775",...,"5,820","4,007","4,345","7,838","6,722","8,541","7,162","10,708","2,086","2,451"
8,Population,Population and dwellings,Census Profile 98-316-X2016001,Land area in square kilometres,630.2,7.41,7.83,4.95,2.81,9.98,...,2.91,5.53,12.31,1.6,1.17,1.68,1.65,1.17,13.23,6.04
9,Population,Age characteristics,Census Profile 98-316-X2016001,Children (0-14 years),"398,135","3,840","3,075","1,760","2,360","3,605",...,"1,785","3,555","9,625","2,325","1,165","1,860","1,800","1,210","4,045","1,960"


In [3]:
population_df = profile_df[(profile_df['Category'] == 'Population') | 
                           (profile_df['Category'] == 'Neighbourhood Information')]
population_df.drop(columns=['Category', 'Topic', 'Data Source'], inplace=True)
population_df.set_index('Characteristic', drop=True, inplace=True)
population_df = population_df.T
population_df.head(10)


C:\Users\A\.virtualenvs\Coursera_Capstone-zZP0SJV7\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Female: 55 to 59 years,Female: 60 to 64 years,Female: 65 to 69 years,Female: 70 to 74 years,Female: 75 to 79 years,Female: 80 to 84 years,Female: 85 to 89 years,Female: 90 to 94 years,Female: 95 to 99 years,Female: 100 years and over
City of Toronto,NaN,NaN,"2,731,571","2,615,060",4.50%,"1,179,057","1,112,929","4,334",630.2,"398,135",...,"94,660","81,600","70,180","51,285","43,430","34,965","25,135","13,500","3,475",650
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,1165,1070,985,690,575,485,350,160,60,10
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,915,795,690,450,405,350,205,100,20,0
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,485,400,325,210,180,210,130,70,5,5
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,915,940,950,700,565,425,345,260,90,25
Banbury-Don Mills,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605",...,1005,895,955,790,730,650,615,360,105,20
Bathurst Manor,34,No Designation,"15,873","15,434",2.80%,"6,418","6,089","3,377",4.7,"2,325",...,605,470,415,275,280,285,265,165,45,10
Bay Street Corridor,76,No Designation,"25,797","19,348",33.30%,"18,436","15,074","14,097",1.83,"1,695",...,500,425,445,320,250,170,135,50,5,0
Bayview Village,52,No Designation,"21,396","17,671",21.10%,"10,111","9,532","4,195",5.1,"2,415",...,775,660,585,440,355,315,230,115,25,0
Bayview Woods-Steeles,49,No Designation,"13,154","13,530",-2.80%,"4,895","4,698","3,240",4.06,"1,515",...,525,475,520,400,400,330,265,180,55,10


In [4]:
population_df.dropna(axis=0, inplace=True)
population_df.head()

Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Female: 55 to 59 years,Female: 60 to 64 years,Female: 65 to 69 years,Female: 70 to 74 years,Female: 75 to 79 years,Female: 80 to 84 years,Female: 85 to 89 years,Female: 90 to 94 years,Female: 95 to 99 years,Female: 100 years and over
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,1165,1070,985,690,575,485,350,160,60,10
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,915,795,690,450,405,350,205,100,20,0
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,485,400,325,210,180,210,130,70,5,5
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,915,940,950,700,565,425,345,260,90,25
Banbury-Don Mills,42,No Designation,"27,695","26,918",2.90%,"12,473","12,124","2,775",9.98,"3,605",...,1005,895,955,790,730,650,615,360,105,20


In [5]:
population_df.shape

(140, 57)

In [6]:
population_df.columns

Index(['Neighbourhood Number', 'TSNS2020 Designation', 'Population, 2016',
       'Population, 2011', 'Population Change 2011-2016',
       'Total private dwellings',
       'Private dwellings occupied by usual residents',
       'Population density per square kilometre',
       'Land area in square kilometres', 'Children (0-14 years)',
       'Youth (15-24 years)', 'Working Age (25-54 years)',
       'Pre-retirement (55-64 years)', 'Seniors (65+ years)',
       'Older Seniors (85+ years)', 'Male: 0 to 04 years',
       'Male: 05 to 09 years', 'Male: 10 to 14 years', 'Male: 15 to 19 years',
       'Male: 20 to 24 years', 'Male: 25 to 29 years', 'Male: 30 to 34 years',
       'Male: 35 to 39 years', 'Male: 40 to 44 years', 'Male: 45 to 49 years',
       'Male: 50 to 54 years', 'Male: 55 to 59 years', 'Male: 60 to 64 years',
       'Male: 65 to 69 years', 'Male: 70 to 74 years', 'Male: 75 to 79 years',
       'Male: 80 to 84 years', 'Male: 85 to 89 years', 'Male: 90 to 94 years',
       

In [10]:
pop_df = population_df[['Neighbourhood Number', 'Population, 2016', 'Population Change 2011-2016', 'Working Age (25-54 years)']]

In [12]:
pop_df.head(10).to_markdown()

'|                              |   Neighbourhood Number | Population, 2016   | Population Change 2011-2016   | Working Age (25-54 years)   |\n|:-----------------------------|-----------------------:|:-------------------|:------------------------------|:----------------------------|\n| Agincourt North              |                    129 | 29,113             | -3.90%                        | 11,305                      |\n| Agincourt South-Malvern West |                    128 | 23,757             | 8.00%                         | 9,965                       |\n| Alderwood                    |                     20 | 12,054             | 1.30%                         | 5,220                       |\n| Annex                        |                     95 | 30,526             | 4.60%                         | 15,040                      |\n| Banbury-Don Mills            |                     42 | 27,695             | 2.90%                         | 10,810                      |\n| Bat

In [7]:
def getNeighborhoods(shape_data):
    neighborhoods = []
    for place in shape_data['features']:
        neighborhood = {'Neighborhood_number': place['properties']['AREA_SHORT_CODE'],
                        'Neighborhood_name': place['properties']['AREA_NAME']}
        coords = np.array(place['geometry']['coordinates'][0])
        centroid = np.mean(coords, axis=0)
        neighborhood['Latitude'] = centroid[1]
        neighborhood['Longitude'] = centroid[0]
        neighborhoods.append(neighborhood)
    return pd.DataFrame(neighborhoods)
        

In [8]:
path = os.path.join(os.path.abspath('../data'), 'Neighbourhoods.geojson')
with open(path, 'r') as FILE:
    data = json.load(FILE)

df_neighborhoods = getNeighborhoods(data)
df_neighborhoods.head()

,Neighborhood_number,Neighborhood_name,Latitude,Longitude
0,96,Casa Loma (96),43.680555,-79.406011
1,95,Annex (95),43.673365,-79.402468
2,109,Caledonia-Fairbank (109),43.688206,-79.459095
3,64,Woodbine Corridor (64),43.675813,-79.314969
4,103,Lawrence Park South (103),43.714161,-79.407022


In [9]:
df_neighborhoods.head(10).to_markdown()

'|    |   Neighborhood_number | Neighborhood_name                 |   Latitude |   Longitude |\n|---:|----------------------:|:----------------------------------|-----------:|------------:|\n|  0 |                    96 | Casa Loma (96)                    |    43.6806 |    -79.406  |\n|  1 |                    95 | Annex (95)                        |    43.6734 |    -79.4025 |\n|  2 |                   109 | Caledonia-Fairbank (109)          |    43.6882 |    -79.4591 |\n|  3 |                    64 | Woodbine Corridor (64)            |    43.6758 |    -79.315  |\n|  4 |                   103 | Lawrence Park South (103)         |    43.7142 |    -79.407  |\n|  5 |                   130 | Milliken (130)                    |    43.8174 |    -79.2763 |\n|  6 |                    53 | Henry Farm (53)                   |    43.7706 |    -79.3437 |\n|  7 |                    26 | Downsview-Roding-CFB (26)         |    43.7332 |    -79.4949 |\n|  8 |                     6 | Kingsview Village-

In [ ]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='Toronto_Explorer')
location = geolocator.geocode(address)
TORONTO_LAT = location.latitude
TORONTO_LONG = location.longitude
print(f'Geographical coordinates of Tornoto = {TORONTO_LAT}, {TORONTO_LONG}')

In [ ]:
toronto_map = folium.Map(location=[TORONTO_LAT, TORONTO_LONG], zoom_start=11)

geo_path = os.path.join(os.path.abspath('../data'), 'Neighbourhoods.geojson')
toronto_map.choropleth(geo_path,
                       key_on='features.properties.AREA_LONG_CODE')

# Neighborhoods
for lat, long, label in zip(df_neighborhoods['Latitude'], 
                            df_neighborhoods['Longitude'],
                            df_neighborhoods['Neighborhood_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, long],
                        popup=label,
                        color='red',
                        radius=3).add_to(toronto_map)

toronto_map

In [ ]:
df_neighborhoods.shape